In [ ]:
# lr, size, range, algo

import torch, random
import torch.nn as nn
import torch.optim as optim


class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(2, 6)
        self.fc2 = nn.Linear(6, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

class SeqModel(nn.Module):
    def __init__(self, input_features, output_features, hidden_units=8):
      super(SeqModel, self).__init__()
      self.linear_layer_stack = nn.Sequential(
            nn.Linear(input_features, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_features),
        )

    def forward(self, x):
      return self.linear_layer_stack(x)


In [ ]:
#X_train = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
#y_train = torch.tensor([[0.0], [1.0], [1.0], [0.0]])

X_trn = [[random.uniform(0, 1), random.uniform(0, 1)] for _ in range(3000)]
y_trn = [[round(X_trn[i][0])^round(X_trn[i][1])] for i in range(3000)]
X_train = torch.tensor(X_trn, dtype=torch.float32)
y_train = torch.tensor(y_trn, dtype=torch.float32)
print(X_train[:10], y_train[:10])

# Instantiate the Model, Define Loss Function and Optimizer
model = SimpleNN()
#criterion = nn.MSELoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

for epoch in range(500):
    model.train()

    # Forward pass
    outputs = model(X_train)
    #print(outputs)
    loss = criterion(outputs, y_train)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/500], Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
    test_data = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
    predictions = model(test_data)
    print(f'Predictions:\n{predictions}')
    # results very sensitive to random initial parameters


# Instantiate the Model, Define Loss Function and Optimizer
model_seq = SeqModel(2,1)
#criterion_seq = nn.MSELoss()
criterion_seq = nn.BCEWithLogitsLoss()
optimizer_seq = optim.Adam(model_seq.parameters(), lr=0.01)

for epoch in range(500):
    model_seq.train()

    # Forward pass
    outputs_seq = model_seq(X_train)
    loss_seq = criterion_seq(outputs_seq, y_train)

    # Backward pass and optimize
    optimizer_seq.zero_grad()
    loss_seq.backward()
    optimizer_seq.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/500], Loss: {loss_seq.item():.4f}')

model_seq.eval()
with torch.no_grad():
    test_data_seq = torch.tensor([[0.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 1.0]])
    predictions_seq = model_seq(test_data_seq)
    print(f'Predictions:\n{predictions_seq}')
    # results very sensitive to random initial parameters

tensor([[0.4147, 0.7176],
        [0.9306, 0.3451],
        [0.1739, 0.4533],
        [0.0716, 0.4903],
        [0.3511, 0.0151],
        [0.4660, 0.3103],
        [0.6940, 0.1621],
        [0.5373, 0.3260],
        [0.1764, 0.7444],
        [0.0380, 0.9287]]) tensor([[1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.]])
Epoch [100/500], Loss: 0.6352
Epoch [200/500], Loss: 0.5883
Epoch [300/500], Loss: 0.5687
Epoch [400/500], Loss: 0.5318
Epoch [500/500], Loss: 0.5248
Predictions:
tensor([[1.2508e-09],
        [1.0000e+00],
        [1.0000e+00],
        [2.4890e-09]])
Epoch [100/500], Loss: 0.5159
Epoch [200/500], Loss: 0.3899
Epoch [300/500], Loss: 0.3755
Epoch [400/500], Loss: 0.3745
Epoch [500/500], Loss: 0.3744
Predictions:
tensor([[-3.4288],
        [ 7.4553],
        [ 7.4015],
        [-3.0542]])


In [1]:
print(model(torch.tensor([[0.52,0.01]])))

NameError: name 'model' is not defined

In [ ]:
class Generator(nn.Module):
    def __init__(self, noise_dim):
        super(Generator, self).__init__()
        self.noise_dim = noise_dim
        self.main = nn.Sequential(
            nn.Linear(noise_dim, 4),
            nn.ReLU(True),
            nn.Linear(4, 2)
        )

    def forward(self, x):
        return self.main(x)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
NOISE_DIM = 4

generator = Generator(NOISE_DIM)
discriminator = Discriminator()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
generator = generator.to(device)
discriminator = discriminator.to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()

generator_optimizer = optim.Adam(generator.parameters(), lr=0.02, betas=(0.5, 0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr=0.02, betas=(0.5, 0.999))

NUM_EPOCHS = 200
BATCH_SIZE = 256
SAMPLE_SIZE = 10000

x_trn = [[random.uniform(0, 1), random.uniform(0, 1)] for _ in range(SAMPLE_SIZE)]
y_trn = [round(x_trn[i][0])^round(x_trn[i][1]) for i in range(SAMPLE_SIZE)]
train_dataset =  [[x_trn[i][0],x_trn[i][1],y_trn[i]] for i in range(SAMPLE_SIZE)]
train_dataset = torch.tensor(train_dataset)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
# malfunctioning, attempting ideas: discriminator~basic dnn with gan attacked

for epoch in range(NUM_EPOCHS):
    for test in train_loader:

      real_images = test[:,:2]
      #real_images = real_images.to(device)

      discriminator_optimizer.zero_grad()
      #real_labels = torch.ones(real_images.size(0), 1, device=device)
      real_labels = test[:,2:]
      #print(real_labels)
      real_outputs = discriminator(real_images)
      real_loss = criterion(real_outputs, real_labels)
      #real_loss.backward()

      noise = torch.randn(real_images.size(0), NOISE_DIM, device=device)
      #print(noise)
      fake_images = generator(noise)  # extending domain to negative side in normality
      real_list = real_labels.tolist()
      fake_labels = torch.tensor([[round(real_list[i][0])^1] for i in range(len(real_list))], dtype=torch.float32)
      fake_outputs = discriminator(fake_images.detach())
      #print(fake_labels)

      #fo_tl = fake_outputs.tolist()
      #fake_outputs_labels = torch.tensor([[round(fo_tl[i][0])] for i in range(len(fo_tl))], dtype=torch.float32)
      fake_loss = criterion(fake_outputs, fake_labels)
      #print(fake_outputs_labels)
      #fake_loss.backward()
      d_loss = (real_loss + fake_loss) / 2
      d_loss.backward()
      discriminator_optimizer.step()

      generator_optimizer.zero_grad()
      #fake_labels = real_labels
      gen_images = generator(noise)
      gen_outputs = discriminator(gen_images)
      gen_loss = criterion(gen_outputs, real_labels)
      gen_loss.backward()
      generator_optimizer.step()

    if epoch == 0 or epoch == NUM_EPOCHS-1:
      print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')
    elif epoch % 10 == 0 and epoch != 0:
      print(f'Epoch [{epoch}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')

Epoch [1/200]Discriminator Loss: 0.3648, Generator Loss: 0.6941
Epoch [10/200]Discriminator Loss: 0.3503, Generator Loss: 0.7814
Epoch [20/200]Discriminator Loss: 0.3697, Generator Loss: 0.9940
Epoch [30/200]Discriminator Loss: 0.3503, Generator Loss: 0.7466
Epoch [40/200]Discriminator Loss: 0.3507, Generator Loss: 0.8587
Epoch [50/200]Discriminator Loss: 0.3754, Generator Loss: 0.6942
Epoch [60/200]Discriminator Loss: 0.3643, Generator Loss: 0.9045
Epoch [70/200]Discriminator Loss: 0.3836, Generator Loss: 1.2288
Epoch [80/200]Discriminator Loss: 0.3563, Generator Loss: 0.6946
Epoch [90/200]Discriminator Loss: 0.3571, Generator Loss: 0.7379
Epoch [100/200]Discriminator Loss: 0.3229, Generator Loss: 1.4454
Epoch [110/200]Discriminator Loss: 0.3595, Generator Loss: 0.6940
Epoch [120/200]Discriminator Loss: 0.3728, Generator Loss: 0.7154
Epoch [130/200]Discriminator Loss: 0.3687, Generator Loss: 0.7189
Epoch [140/200]Discriminator Loss: 0.3472, Generator Loss: 0.8608
Epoch [150/200]Discri

In [ ]:
def generate_and_save_images(model, dis_model, epoch, noise):
    model.eval()
    with torch.no_grad():
        fake_images = model(noise).cpu()
        print(noise)
        print(fake_images)
        print(dis_model(fake_images))
test_noise = torch.randn(4, NOISE_DIM, device=device)
generate_and_save_images(generator, discriminator, NUM_EPOCHS, test_noise)

tensor([[ 1.3549, -0.6994,  1.0298,  0.5693],
        [-1.6062,  0.5869,  1.5516, -0.4347],
        [-0.4399, -0.9809, -0.5423,  0.8993],
        [ 2.8288,  0.5911,  0.2908, -1.4706]])
tensor([[ 1.1218, -3.7041],
        [ 1.1218, -3.7041],
        [ 1.1218, -3.7041],
        [ 1.1218, -3.7041]])
tensor([[0.8984],
        [0.8984],
        [0.8984],
        [0.8984]])


In [ ]:
print(generator(torch.randn(1, NOISE_DIM, device=device)))
print(discriminator(torch.tensor([[0.83,0.13]])))

tensor([[ 1.1218, -3.7041]], grad_fn=<AddmmBackward0>)
tensor([[-67.9189]], grad_fn=<AddmmBackward0>)


In [ ]:
# theoretical gan: simply generate images and making the discriminator considering fake as real

for epoch in range(NUM_EPOCHS):
    for test in train_loader:

      real_signal = test[:,:2]
      #real_images = real_images.to(device)

      real_labels = torch.ones(real_signal.size(0), 1, device=device)
      fake_labels = torch.zeros(real_signal.size(0), 1, device=device)

      discriminator_optimizer.zero_grad()
      noise = torch.randn(real_signal.size(0), NOISE_DIM, device=device)

      #real_labels = test[:,2:]
      #print(real_labels)
      real_outputs = discriminator(real_signal)
      real_loss = criterion(real_outputs, real_labels)
      #real_loss.backward()

      #print(noise)
      fake_signal = generator(noise)
      #real_list = real_labels.tolist()
      #fake_labels = torch.tensor([[round(real_list[i][0])^1] for i in range(len(real_list))], dtype=torch.float32)
      fake_outputs = discriminator(fake_signal)
      #print(fake_labels)

      #fo_tl = fake_outputs.tolist()
      #fake_outputs_labels = torch.tensor([[round(fo_tl[i][0])] for i in range(len(fo_tl))], dtype=torch.float32)
      fake_loss = criterion(fake_outputs, fake_labels)
      #print(fake_outputs_labels)
      #fake_loss.backward()
      d_loss = (real_loss + fake_loss) / 2
      d_loss.backward()
      discriminator_optimizer.step()

      generator_optimizer.zero_grad()
      gen_signal = generator(noise)
      gen_outputs = discriminator(gen_signal)
      gen_loss = criterion(gen_outputs, real_labels)
      gen_loss.backward()
      generator_optimizer.step()

    if epoch == 0 or epoch == NUM_EPOCHS-1:
      print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')
    elif epoch % 10 == 0 and epoch != 0:
      print(f'Epoch [{epoch}/{NUM_EPOCHS}]'#, Step [{epoch+1}/{len(train_loader)}], '
            f'Discriminator Loss: {d_loss.item():.4f}, '
            f'Generator Loss: {gen_loss.item():.4f}')

Epoch [1/200]Discriminator Loss: 4.6938, Generator Loss: 8.4557
Epoch [10/200]Discriminator Loss: 0.6689, Generator Loss: 0.7763
Epoch [20/200]Discriminator Loss: 0.4514, Generator Loss: 0.9949
Epoch [30/200]Discriminator Loss: 0.7657, Generator Loss: 0.7656
Epoch [40/200]Discriminator Loss: 0.4281, Generator Loss: 1.3071
Epoch [50/200]Discriminator Loss: 0.2693, Generator Loss: 1.7138
Epoch [60/200]Discriminator Loss: 0.6284, Generator Loss: 0.8359
Epoch [70/200]Discriminator Loss: 0.4112, Generator Loss: 1.3021
Epoch [80/200]Discriminator Loss: 0.2332, Generator Loss: 1.5395
Epoch [90/200]Discriminator Loss: 0.3241, Generator Loss: 1.7197
Epoch [100/200]Discriminator Loss: 0.4626, Generator Loss: 0.9129
Epoch [110/200]Discriminator Loss: 0.6193, Generator Loss: 1.2024
Epoch [120/200]Discriminator Loss: 0.5965, Generator Loss: 1.1177


In [ ]:
def generate_and_save_images(model, dis_model, epoch, noise):
    model.eval()
    with torch.no_grad():
        fake_images = model(noise).cpu()
        print(noise)
        print(fake_images)
        print(dis_model(fake_images))
test_noise = torch.randn(4, NOISE_DIM, device=device)
generate_and_save_images(generator, discriminator, NUM_EPOCHS, test_noise)

tensor([[ 0.7422, -0.0878, -1.3394, -1.7695],
        [ 0.4495,  2.0385,  0.2653, -0.2018],
        [-0.0385, -0.0175,  1.9948, -0.2122],
        [ 0.2330,  0.5247,  0.2079, -0.4859]])
tensor([[0.3880, 0.3926],
        [0.3880, 0.3926],
        [0.3880, 0.3926],
        [0.3880, 0.3926]])
tensor([[0.3112],
        [0.3112],
        [0.3112],
        [0.3112]])


In [ ]:
print(model(torch.tensor([[0.0,0.8]])))

tensor([[0.9768]], grad_fn=<SigmoidBackward0>)
